This program intends to optimize a portfolio utilising Markowitz Portfolio Theory

Note that you only need to run this program up to cell 6, as the remaining cells are for visual display rather than essential computations.

Start by...

1. Retrieving data from Yahoo! Finance
2. Clean isolate close prices and calculate basic asset metrics
3. Define the covariance function w/constraints and minimize
4. Plot the efficient frontier
5. Plot the capital allocation line

Once this is done, you can copy your portfolio weights and use them in my backtesting program.
(You will also need to input your assets again)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
"""
Enter tickers as such: 
tickers = 'aapl cmg jpm'
"""

tickers = "YOUR ASSETS HERE"
data = yf.download(tickers, period="3y", interval="1d")

"""We only need the adjusted close data; we can drop the rest"""
close = data.loc[:, "Adj Close"]
rtns = close.pct_change()
n = len(close.columns)
# input your starting equity
equity = 10000.00

In [ ]:
"""Calculate asset stats"""
mean_rtn = rtns.mean() * 252 ** 0.5
sd = np.std(rtns)
var = sd ** 2

In [ ]:
"""Define the porfolio functions"""


def covar(x):
    mtx = np.cov(rtns.dropna(), rowvar=False)
    for i in range(n):
        for j in range(n):
            mtx[i, j] = mtx[i, j] * x[i] * x[j]

    return mtx.sum() * 252 ** 0.5


def pofo_return(x, mean_rtn):
    return sum(x * mean_rtn)


def sharpe(x, mean_rtn):
    sd = covar(x) ** 0.5
    rtn = pofo_return(x, mean_rtn)

    return rtn / sd


def cal(x, sr):
    return x * sr


"""This constraint makes sure the weights add up to 100%"""


def constraint1(x):
    sum_weights = 1.0
    for i in range(n):
        sum_weights = sum_weights - x[i]

    return sum_weights


"""This constraint is used for finding efficient frontier values"""


def constraint2(x, target):
    p_rtn = target
    for i in range(n):
        p_rtn = p_rtn - (x[i] * mean_rtn[i])

    return p_rtn


"""This limits the St Dev when finding max return"""


def constraint3(x):
    p_sd = 0.5

    return p_sd - (covar(x) ** 0.5)


con1 = {"type": "eq", "fun": constraint1}
con2 = {"type": "eq", "fun": constraint2}
con3 = {"type": "eq", "fun": constraint3}
cons = [con1, con2, con3]

In [ ]:
x0 = [1 / n for i in range(n)]
"""Fill in list below for manual guesses"""
# x0 = []

"""Find weights for min-variance portfolio"""
opt_var = minimize(covar, x0, method="SLSQP", tol=1e-10, constraints=con1)
min_var_w = opt_var.x

"""Find weights for max-return portfolio"""
opt_rtn = minimize(
    pofo_return, x0, method="SLSQP", args=-mean_rtn, tol=1e-10, constraints=[con1, con3]
)
max_rtn_w = opt_rtn.x

"""Find weights for maximum Sharpe Ratio"""
# these are the actual weights to optimize the portfolio
opt_sr = minimize(
    sharpe, x0, method="SLSQP", args=-mean_rtn, tol=1e-10, constraints=con1
)
max_sr_w = opt_sr.x

In [ ]:
p_var = covar(max_sr_w)
p_sd = p_var ** 0.5
p_rtn = pofo_return(max_sr_w, mean_rtn)

# display weights cohesively
print('Optimized Portfolio Weights\n')
for i in range(n):
    print(
        f"{close.columns[i]}: {round(max_sr_w[i] * 100, 2)}%; ${round(equity * max_sr_w[i], 2)}"
    )

print(f'\nReturn: {round(p_rtn, 2) * 100}%, St Dev: {round(p_sd, 2) * 100}%')

In [ ]:
'''You can copy the weights below for use in my backtest program'''
max_sr_w

Okay great, now we know the optimal weights for our portfolio. We're done optimizing but how can we expand on this?

Charting the minimum-variance frontier will give use a graphical representation of possible portfolio combinations. We can then synthesize this with the capital allocation line to verify that the weights above are accurate.

In [ ]:
"""Calculate Sharpe Ratio; slope of CAL"""
sr = sharpe(max_sr_w, mean_rtn)

"""The following generates weights for the efficient frontier"""
import time

df_values = []
ll = -5
ul = 115
step = 0.0025
start = time.perf_counter()
for i in range(ll, ul):
    target = [i * step]

    con2 = {"type": "eq", "fun": constraint2, "args": target}

    ef_sol = minimize(covar, x0, method="SLSQP", tol=1e-10, constraints=[con1, con2])
    values = ef_sol.x.tolist()
    df_values.append(values)
stop = time.perf_counter()
print(f"Calculated in {round(stop - start, 3)} seconds")
ef_values = pd.DataFrame(
    df_values, columns=close.columns, index=[i for i in range(ll, ul)]
)

In [ ]:
"""This generates data points for the efficient frontier"""
y = []
for i in range(ll, ul):
    value = round(covar(ef_values.loc[i, :]) ** 0.5 * 100, 2)
    y.append(value)

"""This generates points for the capital allocation line"""
calues = []
for i in range(ll, ul):
    value = cal(i, sr)
    calues.append(value)

ef_values["return"] = [i * step * 100 for i in range(ll, ul)]
ef_values["ef"] = y
ef_values["cal"] = calues

In [ ]:
"""Now we want to solve for the intercept of the CAL and efficient frontier"""
# will return to this; it will be used a verification of the max_sr_w
ef_values["int"] = abs(ef_values["cal"] - ef_values["ef"])
idx = ef_values[ef_values["int"] == min(ef_values["int"])].index.values
w = ef_values.iloc[idx - ll, :-4]
# opt_w = []
# for i in range(n):
#     opt_w.append(w.iloc[0, i])
    
'''Plot the efficient frontier and CAL'''
plt.plot(
    ef_values["ef"], ef_values["return"], "b-", ef_values.index, ef_values["cal"], "r-"
)
plt.title("Efficient Frontier")
plt.xlabel("Portfolio SD (%)")
plt.ylabel("Portfolio Return (%)")
plt.grid(True)
plt.xlim(-2, 20)
plt.ylim(-2, 20)
plt.show()

# print(
#     f"{round(p_rtn * 100, 2)}% Return, {round(p_sd * 100, 2)}% St Dev, Sharpe: {round(p_rtn / p_sd, 2)}"
# )

# display weights cohesively
# for i in range(n):
#     print(
#         f"{w.columns[i]}: {round(max_sr_w[i] * 100, 2)}%; ${round(equity * max_sr_w[i], 2)}"
#     )